# Ag-Analytics-Harmonised Landsat Sentinel API-Demo


<br>The Harmonized Landsat and Sentinel-2 (HLS) is a NASA initiative to produce a Virtual Constellation (VC) of surface reflectance (SR) data from the Operational Land Imager (OLI) and MultiSpectral Instrument (MSI) onboard the Landsat 8 and Sentinel-2 remote sensing satellites, respectively. The data from these satellites creates unprecedented opportunities for timely and accurate observation of Earth status and dynamics at moderate (<30 m) spatial resolution every 2-3 days.

<br>The Ag-Analytics provides the service which a user can provide a polygon in specific formats in additional with more customized options to retrieve the dynamics of their land at various times for Landsat from 2013 and Sentinel-2 from 2015 calendar years. Additionally, this service also provides information on cloud cover, statistics, Normalized Difference Vegetation Index in addition to MSI bands information.


<br> Experience the usage of this service on Ag-Analytics website using below link
https://analytics.ag/FarmScope

In [41]:
import requests
import json
import time
import os
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import zipfile, io
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


### Request Parameters Details:

####Request URL: https://ag-analytics.azure-api.net/hls-service/

| Parameter           | Data Type           | Required? | Default                 | Options                                                                                                                       | Description                                                                                                                                          |
|---------------------|---------------------|-----------|-------------------------|-------------------------------------------------------------------------------------------------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------|
| AOI                 | Geometry, file/text | Yes       | -                       | JSON, GEOJSON, Shapefile, Raster                                                                                              | Boundary for area of interest.  |
| Band                | List                | Yes       | -                       | Red, Green, Blue, Coastal Aerosol, NIR, SWIR1, SWIR2, QA, NDVI, RGB, NDWI, NDBI, NDTI, UI, CIR, UE, LW, AP, AGR, FFBS, BE, VW | Provide the list of HLS Spectral band names to retrieve for given AOI. See Figures 3-4 on website page for API.                                                              |
| Startdate           | Date, mm/dd/yyyy    | No        | -                       | -                                                                                                                             | • Landsat – data starts from 2013 •Sentinel – data starts from 2015                                                                                  |
| Enddate             | Date, mm/dd/yyyy    | No        | -                       | -                                                                                                                             | In the absence of startdate or enddate, or both, the service retrieves the latest information available on the land.                                 |
| byweek              | Int, boolean        | No        | 1                       | 1, 0                                                                                                                          | If set to 1, result raster will be the mosaic of all the tiles in a particular week for a given satellite                                            |
| satellite           | text                | No        | Landsat                 | Landsat, Sentinel                                                                                                             | If set to both Landsat, Sentinel then the result raster will be the mosaic of both satellites for the given dates                                    |
| showlatest          | Int, boolean        | No        | 1                       | -                                                                                                                             | If startdate or enddate is not given, shows the latest available tile.                                                                               |
| filter              | Int, boolean        | No       | 0                       | 0, 1                                                                                                                          | If set to 1, the resulting image is cloud filtered with no-data where clouds are.                                                                                  |
| qafilter            | Int, boolean        | No        | 0                       | 0, 1                                                                                                                          | If set to 1, continues to filter tiles until the invalid pixels are < qacloudperc                                                                    |
| qacloudperc         | float               | No        | 100                     | 0-100                                                                                                                         | If qafilter parameter is 1, then filters the tiles until the invalid pixels in those are < qacloudperc |
| displaynormalvalues | float               | No        | 2000                    | -                                                                                                                             | This parameter is used to normalize the band values for display purposes. Used for bands like RGB, AGR, etc.                                         |
| legendtype          | text                | No        | Relative                | Relative, Absolute                                                                                                            | Legend type of display ranges of resulting response.                                                                                                 |
| resolution          | float               | No        | 0.0001                  | -                                                                                                                             | Cellsize in meters.                                                                                                                                  |
| flatten_data        | Int, boolean        | No        | 0                       | 0, 1                                                                                                                          | Flatten data which has a list of Xcoord, Ycoord and Values for each band in the output. If 1, flatten_data is returned.                              |
| statistics          | Int, boolean        | No        | 1                       | 0, 1                                                                                                                          | Returns statistical features of the output .tif file.                                                                                                |
| return_tif          | int                 | No        | 1                       | 0, 1                                                                                                                          | Returns the downloadable link to output raster. If 0, link will not be returned.                                                                     |
| projection          | text                | No        | Projection of AOI Given | See Figure 5.                                                                                                                 | Enter the desired projection for the result raster. See Figure 5 for details.                                                                        |

### Request Parameters

In [58]:
values={"aoi": '{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-92.329005, 41.6850758], [-92.3290479, 41.681366], [-92.3240805, 41.6813179], [-92.3255289, 41.6841784], [-92.329005, 41.6850758]]]}}',
"Band": "['NDVI']",
"Enddate": "3/8/2019",
"Startdate": "3/2/2019",
"legendtype": "Relative",
"qafilter": "1",
"satellite": "Landsat,Sentinel",
"resolution":"0.00001"}
    
headers={'Content-Type':'application/x-www-form-urlencoded'}
           

###  API Function

In [59]:
def harmonisedlandsatdata(values,headers):
    try:
        url = 'https://ag-analytics.azure-api.net/hls-service/'
     
        response = requests.post(url, data=values,headers=headers).json()
        
        print(response)

        return response
    
    except Exception as e:
        print(e)
        raise e

### Calling API Function and Displaying Response

In [60]:
hlsresponse=harmonisedlandsatdata(values,headers)

[{'tiledate': '12/03/2018-12/09/2018', 'band': 'NDVI', 'download_url': 'downloads/raster_bandNDVI_date2018337-2018343_20200309_194756_7821.tif', 'features': [{'attributes': {'OID': 0, 'Extent': '-92.32904454981215, 41.68132515274015, -92.32408454981216, 41.68507515274015', 'Mean': 0.18745804878679692, 'Max': 0.22015209125475285, 'Min': 0.17308613154023203, 'Std': 0.007226297595028316, 'Percentile5': 0.1769952628087829, 'Percentile95': 0.20119802089297484, 'pngb64': 'data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAfAAAAF3CAYAAABaGRmGAAAT4ElEQVR4nO3daXLjRhKA0dKEz8o+k3RZzg+33BSbC5ZaMqvei+gYjx2WKWL5kAAIlgIAAAAAAAAAAAAAAAAAAEAkH/d/43ot1xEv5NbHx9+vCwD4I2TA7wk6APyUIuD3BB2A1aUM+D1BB2A1UwT8nqADMLv/jX4BLVyv5frvn0v6gxEAeGTKCfyPy19/5+Pjy3QOQHrLBfyeoAOQ0fIBvyfoAGQg4G8IOgARCfhOgg5ABAJ+kqADMIKAVyboAPQg4A2JOQCtCHhHgg5ALQI+kKADcNSkAY8d7mcEHYCtJgt4znA/I+gAPDNBwOeK9iuCDsC3hAFfJ9jvCDrAuhIEXLC3EnSAdQQNuGifJeYAcwsUcNFuSdAB5jI44KI9iqAD5DYg4KIdkaAD5NIx4MKdiaADxNY44KI9C0EHiKVRwIV7doIOMFbFgIv2ygQdoK8KARdufhJzgP

### Displaying  Output Image

In [56]:
for resp in hlsresponse:
    if resp["band"]=="NDVI" and resp["download_url"]!="":
        
        print("Legend Values : ",resp["features"][0]['attributes']['Legend'])
        
        img_uri=resp["features"][0]['attributes']["pngb64"]
        
        Image(url=img_uri)

        
        

Image(url=img_uri)        
        

Legend Values :  [{'color': '#ff0000', 'Min': 0.17308613154023203, 'Max': 0.18093045815931882, 'Mean': 0.17700829484977543, 'Area': '15.59 %', 'Count': 19872, 'CountAllPixels': 127477}, {'color': '#ff6666', 'Min': 0.18093045815931882, 'Max': 0.18877478477840565, 'Mean': 0.18485262146886222, 'Area': '48.16 %', 'Count': 61395, 'CountAllPixels': 127477}, {'color': '#ffff66', 'Min': 0.18877478477840565, 'Max': 0.19661911139749244, 'Mean': 0.19269694808794904, 'Area': '25.52 %', 'Count': 32535, 'CountAllPixels': 127477}, {'color': '#ffff00', 'Min': 0.19661911139749244, 'Max': 0.20446343801657924, 'Mean': 0.20054127470703584, 'Area': '7.74 %', 'Count': 9862, 'CountAllPixels': 127477}, {'color': '#66ff66', 'Min': 0.20446343801657924, 'Max': 0.21230776463566603, 'Mean': 0.20838560132612263, 'Area': '2.57 %', 'Count': 3275, 'CountAllPixels': 127477}, {'color': '#00ff00', 'Min': 0.21230776463566603, 'Max': 0.22015209125475285, 'Mean': 0.21622992794520945, 'Area': '0.42 %', 'Count': 538, 'CountAl

# GET Request

GET request for downloading the HLS images returned in the POST request.

In [46]:
# User specified local path where file will be downloaded.
local_path = r"C:\<Path to download directory>" # E.g., r"C:\Users\John_Doe\Documents\rasters"

In [47]:
def HLS_get(local_path):
    try:
        url = "https://ag-analytics.azure-api.net/hls-service/"
    
        for resp in hlsresponse:
            file_url = resp['download_url']
            file_name = file_url.split('/')[-1]
            download_path = os.path.join(local_path, file_name)
            
            values = {'filename': file_name}
     
            response = requests.get(url, params=values)
            open(download_path, 'wb').write(response.content)
        
            print(response.url)

        return response
    
    except Exception as e:
        print(e)
        raise e

In [48]:
HLS_get(local_path)

https://ag-analytics.azure-api.net/hls-service/?filename=raster_bandNDVI_date2018351-2018357_20200309_194409_8306.tif


<Response [200]>